In [1]:
import time
import random
import numpy as np
import torch
import json

import messages
from simulator import Simulator
from agent import Agent
from unity_env import UnityEnv

<class 'sac.sac_agent.SACAgent'>
<class 'sac.sac_env.SACEnv'>


In [16]:
simulator = Simulator(port=5004)

In [17]:
env = UnityEnv(simulator, 9, 2, 1, lambda: 2*np.random.random((2,))-1, "SimpleUnityEnv")
agent = Agent(env)

In [ ]:
cp = torch.load(f"save/{env.get_name()}/checkpoint.pt")
agent.policy.load_state_dict(cp['pi'])
agent.q1.load_state_dict(cp['q1'])
agent.q2.load_state_dict(cp['q2'])
agent.replay_buffer = cp["replay_buffer"]
from_epoch = cp['epoch']

In [18]:
simulator.start_server()
serv = simulator.server

Accepted connection from <server.Client object at 0x7fd1959d56f0>


In [19]:
env.set_time_scale(20)

In [20]:
agent.train(from_epoch=0, epoch_len=2000, start_steps=4000)

=== epoch 0 || avg rew : 0.07539238929748535 || avg len : 78.6
=== epoch 1 || avg rew : 7.20237922668457 || avg len : 109.2


In [ ]:
msg = messages.RequestMessage("test", "").to_json()
msg2 = messages.RequestMessage("timeScale", messages.SingleFieldMessage(2.).to_json()).to_json()
msg3 = messages.RequestMessage("pause", messages.SingleFieldMessage(False).to_json()).to_json()
msg4 = messages.RequestMessage("reset", "").to_json()
msg5 = messages.RequestMessage("step", messages.ControllMessage(5, 5).to_json()).to_json()


In [52]:
r = lambda d: {k:round(v, 5) for (k, v) in d.items()}

print(r(simulator._reset().velocity))
time.sleep(env.get_target_frame_duration())
print(r(simulator._step(10, 1).velocity))
for _ in range(3):
    #time.sleep(env.get_target_frame_duration())
    print(r(simulator._step(0, 1).velocity))
simulator.shutdown()
simulator.server.clients = []

IndexError: list index out of range

Accepted connection from <server.Client object at 0x7fd6a5d6ffa0>


In [10]:
serv.stop_server()

In [7]:
simulator.shutdown()
simulator.server.clients = []

Accepted connection from <server.Client object at 0x7fd11cc34d60>


In [55]:
simulator.server.clients = []

Accepted connection from <server.Client object at 0x7fd5cab9b2e0>
